# 모델 저장하기

## 훈련하는 동안 체크포인트 저장하기

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
# 단순한 모델 정의
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

In [5]:
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#### 체크포인트 콜백 사용하기

In [6]:
# 체크포인트 파일 경로 설정
checkpoint_path = "training_1/cp.ckpt" 
checkpoint_dir = os.path.dirname(checkpoint_path) # 현재 경로 폴더 반환
checkpoint_dir

'training_1'

In [7]:
# 모델의 가중치를 저장하는 체크포인트 콜백 선언
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백 전달

Epoch 1/10
24/32 [=====================>........] - ETA: 0s - loss: 1.3313 - accuracy: 0.6146
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 13ms/step - loss: 1.1836 - accuracy: 0.6540 - val_loss: 0.6997 - val_accuracy: 0.7820
Epoch 2/10
24/32 [=====================>........] - ETA: 0s - loss: 0.4333 - accuracy: 0.8815
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.4185 - accuracy: 0.8840 - val_loss: 0.5281 - val_accuracy: 0.8500
Epoch 3/10
26/32 [=======================>......] - ETA: 0s - loss: 0.2787 - accuracy: 0.9243
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.2832 - accuracy: 0.9250 - val_loss: 0.4744 - val_accuracy: 0.8500
Epoch 4/10
26/32 [=======================>......] - ETA: 0s - loss: 0.2016 - accuracy: 0.9567
Epoch 4: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss

In [ ]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'cp.ckpt.index', 'checkpoint']

 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트합니다  
 - 아키텍처가 동일한 모델 간에는 가중치를 공유 가능 
 
 - 그렇다면, 원래 모델과 동일한 아키텍처를 가진 모델을 만든 다음 체크포인트 부여해보자.

In [ ]:
# 동일한 아키텍처를 가진 모델 선언
model = create_model()

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3286 - accuracy: 0.0750 - 264ms/epoch - 8ms/step
훈련되지 않은 모델의 정확도:  7.50%


허접하다.

In [ ]:
# 체크포인트에서 가중치를 로드하고 평가해보자.
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4056 - accuracy: 0.8700 - 82ms/epoch - 3ms/step
복원된 모델의 정확도: 87.00%


In [ ]:
# 파일 이름에 epoch 값 부여 (format 형식)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

- 체크포인트 주요 파라미터 

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

In [ ]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [ ]:
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 15ms/step - loss: 1.1372 - accuracy: 0.6920 - val_loss: 0.7003 - val_accuracy: 0.8040
Epoch 2/50
32/32 [==============================] - 0s 8ms/step - loss: 0.4218 - accuracy: 0.8760 - val_loss: 0.5486 - val_accuracy: 0.8360
Epoch 3/50
32/32 [==============================] - 0s 10ms/step - loss: 0.2969 - accuracy: 0.9170 - val_loss: 0.4676 - val_accuracy: 0.8490
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2086 - accuracy: 0.9520 - val_loss: 0.4341 - val_accuracy: 0.8570
Epoch 5/50
32/32 [==============================] - 0s 10ms/step - loss: 0.1553 - accuracy: 0.9590 - val_loss: 0.4618 - val_accuracy: 0.8460
Epoch 6/50
32/32 [==============================] - 0s 8ms/step - loss: 0.1157 - accuracy: 0.9800 - val_loss: 0.4118 - val_accuracy: 0.8530
Epoch 7/50
32/32 [==============================] - 0s 8ms/step - loss: 0.0846 - accuracy: 0.9910 - val_loss: 0.4097 - val_accuracy: 0.8660
Epoch 8/50
32/32

In [ ]:
# 동일한 아키텍처를 가진 모델 선언
model = create_model()

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3747 - accuracy: 0.1060 - 254ms/epoch - 8ms/step
훈련되지 않은 모델의 정확도: 10.60%


In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0011.ckpt'

In [ ]:
model.load_weights(latest)

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3971 - accuracy: 0.8640 - 114ms/epoch - 4ms/step
훈련되지 않은 모델의 정확도: 86.40%


In [ ]:
### mytraining1 폴더에 저장
### best only로 저장, 최종 값만 저장되도록
### weight값만
checkpoint_path = "mytraining1/cp.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

model = create_model()

model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 13ms/step - loss: 1.1818 - accuracy: 0.6580 - val_loss: 0.7719 - val_accuracy: 0.7610
Epoch 2/50
32/32 [==============================] - 0s 11ms/step - loss: 0.4324 - accuracy: 0.8770 - val_loss: 0.5414 - val_accuracy: 0.8250
Epoch 3/50
32/32 [==============================] - 0s 8ms/step - loss: 0.2903 - accuracy: 0.9160 - val_loss: 0.4786 - val_accuracy: 0.8500
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2074 - accuracy: 0.9530 - val_loss: 0.4244 - val_accuracy: 0.8710
Epoch 5/50
32/32 [==============================] - 0s 8ms/step - loss: 0.1527 - accuracy: 0.9690 - val_loss: 0.4512 - val_accuracy: 0.8560
Epoch 6/50
32/32 [==============================] - 0s 11ms/step - loss: 0.1198 - accuracy: 0.9770 - val_loss: 0.4106 - val_accuracy: 0.8620
Epoch 7/50
32/32 [==============================] - 0s 10ms/step - loss: 0.0868 - accuracy: 0.9880 - val_loss: 0.4240 - val_accuracy: 0.8590
Epoch 8/50
32/3

In [ ]:
model.load_weights(checkpoint_path)

model.evaluate(test_images,  test_labels, verbose=2)

32/32 - 0s - loss: 0.4073 - accuracy: 0.8690 - 88ms/epoch - 3ms/step


[0.4072527587413788, 0.8690000176429749]

In [ ]:
tf.keras.callbacks.ModelCheckpoint?

In [ ]:
### mytraining2 폴더에 저장
### 5epoch마다 저장
### weight값만
checkpoint_path = "mytraining3/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=False, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    period=5, # 체크포인트 저장하는 에폭주기 설정 가능
    save_freq='epoch' # epoch의 끝에서 저장할지 여부
    )

In [ ]:
model = create_model()

model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 15ms/step - loss: 1.1292 - accuracy: 0.6870 - val_loss: 0.7308 - val_accuracy: 0.7860
Epoch 2/50
32/32 [==============================] - 0s 10ms/step - loss: 0.4274 - accuracy: 0.8940 - val_loss: 0.5182 - val_accuracy: 0.8380
Epoch 3/50
32/32 [==============================] - 0s 8ms/step - loss: 0.2943 - accuracy: 0.9220 - val_loss: 0.4921 - val_accuracy: 0.8430
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2146 - accuracy: 0.9450 - val_loss: 0.4232 - val_accuracy: 0.8700
Epoch 5/50
32/32 [==============================] - 0s 8ms/step - loss: 0.1561 - accuracy: 0.9610 - val_loss: 0.4145 - val_accuracy: 0.8670
Epoch 6/50
32/32 [==============================] - 0s 8ms/step - loss: 0.1159 - accuracy: 0.9780 - val_loss: 0.4023 - val_accuracy: 0.8720
Epoch 7/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0890 - accuracy: 0.9860 - val_loss: 0.4097 - val_accuracy: 0.8720
Epoch 8/50
32/32

### early stopping & 체크포인트 같이 사용하기

In [ ]:
checkpoint_path = "training_3/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [ ]:
model.fit(train_images, 
          train_labels,
          epochs=100, 
          callbacks=[cp_callback, early], # 체크포인트 부여, early stopping
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/100
32/32 [==============================] - 1s 22ms/step - loss: 1.1985 - accuracy: 0.6580 - val_loss: 0.7163 - val_accuracy: 0.7870
Epoch 2/100
32/32 [==============================] - 0s 9ms/step - loss: 0.4353 - accuracy: 0.8760 - val_loss: 0.5276 - val_accuracy: 0.8320
Epoch 3/100
32/32 [==============================] - 0s 9ms/step - loss: 0.2839 - accuracy: 0.9280 - val_loss: 0.4726 - val_accuracy: 0.8450
Epoch 4/100
32/32 [==============================] - 0s 10ms/step - loss: 0.2128 - accuracy: 0.9460 - val_loss: 0.4528 - val_accuracy: 0.8490
Epoch 5/100
32/32 [==============================] - 0s 11ms/step - loss: 0.1452 - accuracy: 0.9740 - val_loss: 0.4229 - val_accuracy: 0.8560
Epoch 6/100
32/32 [==============================] - 0s 11ms/step - loss: 0.1205 - accuracy: 0.9760 - val_loss: 0.4207 - val_accuracy: 0.8590
Epoch 7/100
32/32 [==============================] - 0s 9ms/step - loss: 0.0864 - accuracy: 0.9870 - val_loss: 0.4206 - val_accuracy: 0.8700
Epoch 8/1

### 직접 가중치 체크포인트 저장

In [ ]:
# 가중치 저장
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

# 가중치 복원
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.5552 - accuracy: 0.8770 - 285ms/epoch - 9ms/step
복원된 모델의 정확도: 87.70%


## 전체 모델 저장(모델레이어와 함께 저장)
1) save_weights_only=False 로 변경  
2) 직접 model.save() : SavedModel(pb)  
3) HDF5 파일로 저장하기

### save_weights_only=False 로 변경

In [ ]:
checkpoint_path = "training_4/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=False, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

In [ ]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [ ]:
model.fit(train_images, 
          train_labels,
          epochs=100, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/100
32/32 [==============================] - 2s 38ms/step - loss: 1.1517 - accuracy: 0.6760 - val_loss: 0.7206 - val_accuracy: 0.7840
Epoch 2/100
32/32 [==============================] - 1s 28ms/step - loss: 0.4284 - accuracy: 0.8780 - val_loss: 0.5080 - val_accuracy: 0.8450
Epoch 3/100
32/32 [==============================] - 0s 10ms/step - loss: 0.2829 - accuracy: 0.9280 - val_loss: 0.5088 - val_accuracy: 0.8430
Epoch 4/100
32/32 [==============================] - 1s 26ms/step - loss: 0.2196 - accuracy: 0.9480 - val_loss: 0.4279 - val_accuracy: 0.8570
Epoch 5/100
32/32 [==============================] - 0s 11ms/step - loss: 0.1560 - accuracy: 0.9630 - val_loss: 0.4283 - val_accuracy: 0.8550
Epoch 6/100
32/32 [==============================] - 0s 10ms/step - loss: 0.1240 - accuracy: 0.9730 - val_loss: 0.4310 - val_accuracy: 0.8660
Epoch 7/100
32/32 [==============================] - 0s 8ms/step - loss: 0.0935 - accuracy: 0.9850 - val_loss: 0.4327 - val_accuracy: 0.8710
Epoch 8

### SavedModel(pb)
SavedModel 형식은 protobuf 바이너리와 TensorFlow 체크포인트를 포함

In [ ]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 6ms/step - loss: 1.1638 - accuracy: 0.6820
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4268 - accuracy: 0.8850
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2814 - accuracy: 0.9290
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2026 - accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1475 - accuracy: 0.9670
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
# my_model 디렉토리
!ls saved_model

my_model


In [ ]:
# assests 폴더, saved_model.pb, variables 폴더
!ls saved_model/my_model

assets	keras_metadata.pb  saved_model.pb  variables


In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 512)               401920    
                                                                 
 dropout_10 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있습니다. 이 모델을 평가하고 예측에 사용해 보죠:

In [ ]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4266 - accuracy: 0.8590 - 180ms/epoch - 6ms/step
복원된 모델의 정확도: 85.90%
(1000, 10)


### HDF5 파일로 저장하기
케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다.

In [ ]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1830 - accuracy: 0.6840
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4303 - accuracy: 0.8760
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2904 - accuracy: 0.9290
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2154 - accuracy: 0.9410
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1603 - accuracy: 0.9630


### h5 VS pb 
- 공통점 : 레이어 & 가중치 모두 저장
- 차이점  
1) HDF5 : 객체 구성을 사용하여 모델 아키텍처를 저장  
2) SavedModel : 실행 그래프를 저장. 원본 코드 없이도 사용자 지정 객체를 저장 가능

#### h5 to pb 변환하기

In [ ]:
!mkdir converted

In [ ]:
model = tf.keras.models.load_model('my_model.h5')

In [ ]:
export_path = 'converted'
model.save(export_path, save_format="tf")

INFO:tensorflow:Assets written to: converted/assets
